In [ ]:
from time import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
from string import punctuation
from matplotlib import pyplot as plt
from collections import defaultdict
from nltk.corpus import stopwords
cachedstopwords = stopwords.words('english')
from torch.utils.data.dataset import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
%matplotlib inline

In [ ]:
 print(torch.__version__)

In [ ]:
df = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',engine='python')
df.columns = ['polarity', 'review']

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df.columns

In [ ]:
df['polarity'].unique()

In [ ]:
import random
idx = random.randint(0,df['polarity'].count())
print("IDX :",idx)
print(df.iloc[idx]['review'])

In [ ]:
review_size=list(map(len,df['review']))
max_len = max(review_size)
max_len

In [ ]:
#since max size is only 298 so i am not trimming 
sentence_len_dict=defaultdict(lambda:0)
for i in review_size:
    sentence_len_dict[i]+=1
plt.bar(sentence_len_dict.keys(),sentence_len_dict.values())

In [ ]:
def removestopwords(x):
    return [i.lower() for i in x.split() if i.lower() not in cachedstopwords]
    

In [ ]:
df['p_review'] = df['review']
df['p_review'].replace(to_replace = '[{}]'.format(punctuation),inplace=True,value='',regex=True)
df['p_review'].replace(to_replace = '\s+',inplace=True,value=' ',regex=True)
df['p_review'] = df['p_review'].apply(removestopwords)


In [ ]:
df.head()

In [ ]:
#converting polarity to numnber
df['p_polarity']=df['polarity']
df['p_polarity'] = df['p_polarity'].map({'neutral':2,'positive':1,'negative':0})

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df['length_of_sentence']=df['p_review'].str.len()

In [ ]:
indexNames = df[ df['length_of_sentence'] == 0 ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.count()

In [ ]:
df.head()

In [ ]:
#get all keywords
word_cnt = defaultdict(lambda:0)
word_cnt[' ']=1
for i in df['p_review']:
    for j in i:
        word_cnt[j]+=1
voc=word_cnt.keys()


In [ ]:
plt.bar(list(word_cnt.keys())[:100],list(word_cnt.values())[:100])

In [ ]:
reviews = list(df['p_review'])
polarity=list(df['p_polarity'])
length_of_sentence = list(df['length_of_sentence'])
my_idxs=list(range(0,len(polarity)))
np.random.seed(3)
np.random.shuffle(my_idxs)
divider = round(len(polarity)*.80)

train_dataset = [{'review':reviews[i],'polarity':polarity[i],'length_of_sentence':length_of_sentence[i]} for i in my_idxs[:divider]]
valid_dataset = [{'review':reviews[i],'polarity':polarity[i],'length_of_sentence':length_of_sentence[i]} for i in my_idxs[divider:]]


In [ ]:
print("Train Set: {},Validation Set: {}".format(len(train_dataset),len(valid_dataset)))

In [ ]:
wrd_to_idx=dict()
idx_to_wrd=dict()
for i,j in enumerate(voc):
    wrd_to_idx[j]=i
    idx_to_wrd[i]=j

In [ ]:
epoch =50
batch_sz=16
input_dim=100
hidden_layers=50
stack_layer=2
out_dim=3
lr=0.1


In [ ]:
wrd_to_idx[' ']

In [ ]:
import copy
class MyLoader(Dataset):
    def __init__(self,data_set):
        
        self.data=data_set
        
    def __getitem__(self,index):
        wrd_idx=[]
        
        for idx,word in enumerate(self.data[index]['review']):
            
            wrd_idx.append(wrd_to_idx[word])
        
        return wrd_idx,self.data[index]['polarity'],self.data[index]['length_of_sentence']
        
    def __len__(self):
        return len(self.data)
def my_collate_fn(batch):
    reviews_raw=[i[0] for i in batch]
    polarities_raw=[i[1] for i in batch]
    length_of_sentence_raw=[i[2] for i in batch]
    reviews=[]
    polarities=[]
    length_of_sentence=[]
    #print("Before :",reviews_raw,"\n\n",polarities_raw,"\n\n",length_of_sentence_raw)
    #sorting in decreasing order
    for k1 in np.argsort(np.array(length_of_sentence_raw))[::-1]:
        reviews.append( torch.tensor(reviews_raw[k1], dtype=torch.float))
        polarities.append(polarities_raw[k1])
        length_of_sentence.append(length_of_sentence_raw[k1])
    #print("After :",reviews,"\n\n",polarities,"\n\n",length_of_sentence)
    torch.manual_seed(5)
    emb = nn.Embedding(len(voc), input_dim)
    emb_input = torch.nn.utils.rnn.pad_sequence(reviews,batch_first=True)
    #emb_input = torch.tensor(reviews, dtype=torch.float)
#     emb_input = pad_sequence(emb_input,batch_first=True)
    
#     print(new_reviews)
    embidd = emb(emb_input)
    p_embidd = torch.tensor(polarities, dtype=torch.float)
    length_of_sentence = torch.tensor(length_of_sentence, dtype=torch.float)
    return embidd,p_embidd,length_of_sentence

In [ ]:
train_obj = MyLoader(train_dataset)
train_loader=torch.utils.data.DataLoader(train_obj,batch_size=batch_sz,collate_fn=my_collate_fn)

valid_obj = MyLoader(valid_dataset)
valid_loader=torch.utils.data.DataLoader(valid_obj,batch_size=batch_sz,collate_fn=my_collate_fn)

In [ ]:
device="cpu"
if torch.cuda.is_available():
    device='cuda'
print(device)

In [ ]:
#torch function to padd seq
# second seq is of size .after passing to pad_sequence it will become 5 (it will be size of max swq len)
import torch 

seq_batch = [torch.tensor([[1, 1],
                           [2, 2],
                           [3, 3],
                           [4, 4],
                           [5, 5]]),
             torch.tensor([[10, 10],
                           [20, 20],
                           [20, 20]]),
            torch.tensor([[10, 10],
                           [20, 20]])]

seq_lens = [5,3,2]
padded_seq_batch = torch.nn.utils.rnn.pad_sequence(seq_batch, batch_first=True)
print(type(padded_seq_batch))
print(padded_seq_batch,'\n\n',padded_seq_batch.shape)
packed_seq_batch = torch.nn.utils.rnn.pack_padded_sequence(padded_seq_batch, lengths=seq_lens, batch_first=True)
print("packed_seq_batchpacked_seq_batch:",packed_seq_batch)
print(packed_seq_batch.data.shape)

print("\n\n\n********************Unpacking Sequence********************\n")
# unpack sequence
unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(packed_seq_batch, batch_first=True)
print(unpacked.shape, unpacked_len)

In [ ]:
class SentimentAnlysis(nn.Module):
    def __init__(self,input_dim,hidden_layers,stack_layer,out_dim):
        super(SentimentAnlysis, self).__init__()
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.stack_layer = stack_layer
        self.out_dim = out_dim
        #if input shape is (batch_size, seq_len, features(dimension)) then batch_first should br True
        self.rnn = nn.RNN(self.input_dim,self.hidden_layers,self.stack_layer,batch_first=True)
        self.fc=nn.Linear(self.hidden_layers,self.out_dim)
        
    def forward(self,input_,hidden_,seq_lengths):
        batch_size = input_.shape[0]
        
        #since we are passing batch as first dimension we need to transpose
        #from (batch*len_of_seq*dim)---> (len_of_seq*batch*dim)
        
#         n_input = torch.nn.utils.rnn.pad_sequence(input_, batch_first=True)
        
       
        #Excluding padding during training
        packed_input = pack_padded_sequence(input_, seq_lengths.cpu().numpy(),batch_first=True)
#         print("packed_input :",packed_input.shape)
        
        #input
            #n_input = (seq_len, batch, input_size)
            #hidden_ = (num_layers * num_directions, batch, hidden_size).
        #output
#           is the output value at all time-steps of the RNN layer 
            #out  (seq_len, batch, num_directions * hidden_size)
            
            #is the hidden value at the last time-step RNN layers
            #nxt_hidden (num_layers * num_directions, batch, hidden_size)
            
        out,nxt_hidden = self.rnn(packed_input,hidden_)
        
        
#         print("Input Shape of FC :",out.shape)
#         print(out)
        
        
        unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        
        out = self.fc(unpacked)
#         print("\n\npopopo",out)
#         print("Output Shape of FC :",out.shape)
#         print("-"*30)
        
         # softmax function
#         sft_max_out = self.multi(out)
        
#         # reshape to be batch_size first
#         sft_max_out = sft_max_out.view(batch_size, -1)
#         sft_max_out = sft_max_out[:, -1] # get last batch of labels
        return out,nxt_hidden
    
    def initilizeh(self,batch_size):
        '''
            (num_layers * num_directions, batch, hidden_size)
            
            bidirection is false 
        '''
        h = np.zeros((self.stack_layer,batch_size,self.hidden_layers))
        return torch.tensor(h,dtype=torch.float)

In [ ]:
model = SentimentAnlysis(input_dim,hidden_layers,stack_layer,out_dim)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
print("Running on ",device)
sft = nn.Softmax(dim=1)
for itr in range(0,epoch):
    loss_=0
    correct_cnt=0
    valid_loss=0
    start_time = time()
    ################ Train ###################################################
    model.train()
    for i,(records,polarity,sent_lens) in enumerate(train_loader): 
        hitten = model.initilizeh(records.shape[0])
        optimizer.zero_grad()
        out_t,hid = model(records.to(device),hitten.to(device),sent_lens)
        torch.set_printoptions(profile="full")

        
        out_t = out_t.squeeze()
         # reshape to be batch_size first        
        out_t = out_t[:, -1] # get last batch of labels
#         print("Last time step :",out_t)
#         print("Last time step :",out_t.shape)
#         print("*"*30,"\n\n\n\n")
        d=polarity.unsqueeze(dim=1)
        #print("aaaa",polarity)
#         print(out.squeeze(),"\n\n\n",polarity)
#         o=torch.transpose(out,0,1)
#         loss = criterion(o.to(device),(d.float()).to(device))
#         print(out_t)
        out_t = sft(out_t)
    
#       oss = criterion(output, target_tensor)
#       loss.backward()
#       optimizer.step()
    
        loss = criterion(out_t.to(device),polarity.float().to(device))
        loss_+=loss.item()
        loss.backward()
        optimizer.step()
    ################################# validation ##############################
    model.eval()
    for i,(records,polarity,sent_lens) in enumerate(valid_loader): 
        hitten = model.initilizeh(records.shape[0])
        
        optimizer.zero_grad()
        out,hid = model(records.to(device),hitten.to(device),sent_lens)
#         out_t = torch.transpose(out,0,1)
        #out_t = out_t.squeeze()
        out_t = out.squeeze()
        out_t = out_t[:, -1] # get last batch of labels
        d=polarity.unsqueeze(dim=1)
#         print(out_t)
#         print("\n\n",polarity)
        
        
        out_t = sft(out_t)
        oot = out_t
        #this will return max from each element and its index(for us undex is noting but label i,e 0,1,2)
        _,pred_lab = torch.max(oot.data, 1)
        
        correct = (pred_lab.to(device) == polarity.to(device)).sum()
        correct_cnt+=correct.item()
        
        v_loss = criterion(out_t.to(device),polarity.float().to(device))
        valid_loss+=v_loss.item()
        
        
    end_time = time()
    time_taken = end_time - start_time # time_taken is in seconds
    hours, rest = divmod(time_taken,3600)
    minutes, seconds = divmod(rest, 60)
    
    print("Epoch :",itr+1 ,"Train Loss :",loss_/len(train_dataset),"Validation Loss :",valid_loss/len(valid_dataset)," Accuracy :",correct_cnt/len(valid_dataset)," Time Taken :",hours," hr::",minutes," min::",round(seconds)," ss")
    
#  n_input : torch.Size([25, 16, 100])
# packed_input : torch.Size([202, 100])   

In [ ]:
# Running on  cpu
# Epoch : 1 Train Loss : 0.058593509874036234 Validation Loss : 0.05998079787835986  Accuracy : 0.5758513931888545  Time Taken : 0.0  hr:: 0.0  min:: 11  ss
# Epoch : 2 Train Loss : 0.057905072250673845 Validation Loss : 0.06012444765575161  Accuracy : 0.5779153766769866  Time Taken : 0.0  hr:: 0.0  min:: 9  ss
# Epoch : 3 Train Loss : 0.05750968081720414 Validation Loss : 0.060485478279145265  Accuracy : 0.5768833849329206  Time Taken : 0.0  hr:: 0.0  min:: 11  ss
# Epoch : 4 Train Loss : 0.05733654836685427 Validation Loss : 0.06053810675077763  Accuracy : 0.5717234262125903  Time Taken : 0.0  hr:: 0.0  min:: 7  ss
# Epoch : 5 Train Loss : 0.056865862569501324 Validation Loss : 0.061418353034984954  Accuracy : 0.5717234262125903  Time Taken : 0.0  hr:: 0.0  min:: 10  ss
# Epoch : 6 Train Loss : 0.056385914910224176 Validation Loss : 0.062385602724441436  Accuracy : 0.5717234262125903  Time Taken : 0.0  hr:: 0.0  min:: 10  ss
# Epoch : 7 Train Loss : 0.05605482151431422 Validation Loss : 0.06258670153755645  Accuracy : 0.5737874097007224  Time Taken : 0.0  hr:: 0.0  min:: 10  ss
# Epoch : 8 Train Loss : 0.05585221382879442 Validation Loss : 0.0627989529394636  Accuracy : 0.5706914344685242  Time Taken : 0.0  hr:: 0.0  min:: 10  ss
# Epoch : 9 Train Loss : 0.05556135056095739 Validation Loss : 0.0628178871945085  Accuracy : 0.5748194014447885  Time Taken : 0.0  hr:: 0.0  min:: 10  ss
# Epoch : 10 Train Loss : 0.055119802482666506 Validation Loss : 0.0637615744728053  Accuracy : 0.5717234262125903  Time Taken : 0.0  hr:: 0.0  min:: 7  ss

In [ ]:
x =torch.tensor([
                    [-0.2189, -1.0233,  1.1002],
                    [-1.6892,  0.8920, -0.5510],
                    [-1.4843,  0.1082,  0.9838],
                    [-1.7498, -0.3874,  0.6971],
                    [-1.9330, -0.5803,  0.3496],
                    [-1.7892, -0.3349,  0.6219],
                    [-1.8425, -0.4778,  0.5351],
                    [-1.8895, -0.5115,  0.4298],
                    [-1.8252, -0.4108,  0.5649],
                    [-1.8655, -0.4934,  0.4911],
                    [-1.8687, -0.4787,  0.4773],
                    [-1.8434, -0.4486,  0.5291],
                    [-1.8675, -0.4878,  0.4852],
                    [-1.8595, -0.4676,  0.4975],
                    [-1.8537, -0.4655,  0.5100],
                    [-1.8643, -0.4797,  0.4900]
                ]
               )
print(x.shape)
print([2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2])
sft = nn.Softmax(dim=1)
oot = sft(x)
oot

In [ ]:
#this will return max from each element and its index(for us undex is noting but label i,e 0,1,2)
_,pred_lab = torch.max(oot.data, 1)
pred_lab
# [2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2]

In [ ]:
correct = (pred_lab == torch.tensor([2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2])).sum()
correct

In [ ]:
# loss = nn.CrossEntropyLoss()
# _input = torch.randn(3, 2, requires_grad=True)
# target = torch.empty(3, dtype=torch.float).random_(2)
# output = loss(_input, target)
# output

criterion = nn.CrossEntropyLoss().cuda()
_input = torch.autograd.Variable(torch.randn((3,5)))
tgt = torch.autograd.Variable(torch.LongTensor(3).random_(5))
print(_input,_input.shape)
print(tgt)
loss = criterion(_input,tgt)
loss

In [ ]:
#illustrating fc


In [ ]:
hidden_size=3
batch=3
seq_len=2

T_data = [[[1., 2.,1.], [3., 4.,1.]],
          [[5., 6.,1.], [7., 8.,1.]],
          [[1., 2.,1.], [3., 4.,1.]],
         ]
T = torch.tensor(T_data)

T = T.transpose(0,1)
print("Output of RNN\n\n")
print(T)
print(T.shape)

fc=nn.Linear(hidden_size,1)
z = fc(T)
print("\n\nOutput of FC\n\n")
print(z)
print(z.shape)

In [ ]:
4/2


In [ ]:
# torch.set_printoptions(precision=10)
zz

In [ ]:
np.array([-4.3963e-01,  2.9392e-01,  2.1258e-01,  6.5708e-01,  2.2840e-01,
          -9.8323e-02,  2.0006e-01, -7.0602e-01, -1.2906e-01, -3.7235e-01,
           2.2960e-01,  4.4903e-01, -2.7746e-01,  2.9838e-01,  2.0086e-01,
          -1.4690e-01,  1.8579e-01,  2.9300e-01, -4.0135e-01,  5.7170e-02])

In [ ]:
z=torch.tensor([[[-0.0536],
         [-0.2593],
         [-0.1455],
         [-0.1746],
         [-0.1541],
         [-0.2253],
         [-0.1649],
         [-0.1411]],

        [[-0.1079],
         [-0.1342],
         [ 0.0333],
         [ 0.0532],
         [-0.1459],
         [-0.1847],
         [-0.1338],
         [-0.1503]],

        [[-0.0445],
         [-0.2704],
         [-0.2591],
         [-0.1669],
         [-0.1510],
         [-0.1350],
         [-0.2005],
         [-0.3086]],

        [[-0.0822],
         [-0.2452],
         [-0.1207],
         [-0.0947],
         [-0.0009],
         [-0.1569],
         [-0.2562],
         [-0.1742]],

        [[-0.2207],
         [-0.1804],
         [-0.3316],
         [-0.1162],
         [-0.1132],
         [ 0.0203],
         [-0.1796],
         [ 0.1172]],

        [[-0.2966],
         [-0.0165],
         [-0.1236],
         [ 0.0634],
         [-0.0488],
         [-0.1326],
         [-0.2631],
         [-0.1438]],

        [[-0.1018],
         [-0.1955],
         [-0.2225],
         [-0.2263],
         [-0.0969],
         [-0.2181],
         [-0.3999],
         [-0.5065]],

        [[-0.0724],
         [-0.2387],
         [-0.1735],
         [-0.1789],
         [-0.1884],
         [-0.4258],
         [-0.5052],
         [-0.4451]],

        [[-0.0482],
         [ 0.0920],
         [-0.2783],
         [-0.4123],
         [-0.1829],
         [-0.4774],
         [-0.4721],
         [-0.4481]],

        [[-0.2673],
         [-0.2185],
         [-0.2701],
         [ 0.0193],
         [-0.0071],
         [-0.4872],
         [-0.5006],
         [-0.4814]],

        [[-0.0207],
         [-0.1675],
         [-0.2260],
         [-0.2594],
         [-0.4750],
         [-0.5004],
         [-0.4424],
         [-0.4485]],

        [[-0.2907],
         [-0.0419],
         [-0.2722],
         [-0.4477],
         [-0.4896],
         [-0.4444],
         [-0.4431],
         [-0.4398]],

        [[-0.0739],
         [-0.1254],
         [-0.1756],
         [-0.4678],
         [-0.5194],
         [-0.4574],
         [-0.4483],
         [-0.4375]],

        [[-0.0207],
         [-0.2605],
         [-0.2598],
         [-0.3797],
         [-0.4572],
         [-0.4177],
         [-0.4525],
         [-0.4319]],

        [[-0.0666],
         [ 0.0132],
         [ 0.1536],
         [-0.5091],
         [-0.4919],
         [-0.4865],
         [-0.4357],
         [-0.4427]],

        [[ 0.0191],
         [ 0.2059],
         [-0.3197],
         [-0.4517],
         [-0.5423],
         [-0.4545],
         [-0.4515],
         [-0.4373]],

        [[-0.0814],
         [-0.0510],
         [-0.1913],
         [-0.4528],
         [-0.4998],
         [-0.4540],
         [-0.4453],
         [-0.4391]],

        [[-0.1853],
         [-0.0414],
         [-0.5791],
         [-0.4610],
         [-0.4764],
         [-0.4346],
         [-0.4451],
         [-0.4404]],

        [[-0.2207],
         [-0.4276],
         [-0.4988],
         [-0.4518],
         [-0.4446],
         [-0.4384],
         [-0.4437],
         [-0.4413]],

        [[-0.2964],
         [-0.3543],
         [-0.4941],
         [-0.4301],
         [-0.4492],
         [-0.4347],
         [-0.4449],
         [-0.4405]],

        [[-0.0808],
         [-0.3504],
         [-0.5283],
         [-0.4394],
         [-0.4550],
         [-0.4322],
         [-0.4464],
         [-0.4394]]])
print(z.shape)



In [ ]:
# reshape to be batch_size first
sft_max_out = z.view(8, -1)
# sft_max_out = sft_max_out[:, -1] # get last batch of labels
sft_max_out.shape

In [ ]:

sft_max_out = nn.Softmax(torch.tensor([10,20,30,40]))
sft_max_out

In [ ]:
sft_max_out = sft_max_out[:, -1] # get last batch of labels
sft_max_out

In [ ]:
sum([-0.0536, -0.2593, -0.1455, -0.1746, -0.1541, -0.2253, -0.1649, -0.1411,
           -0.1079, -0.1342,  0.0333,  0.0532, -0.1459, -0.1847, -0.1338, -0.1503,
           -0.0445, -0.2704, -0.2591, -0.1669, -0.1510])

In [ ]:
sum([-0.2966,
           -0.0165,
           -0.1236,
            0.0634,
           -0.0488,
           -0.1326,
           -0.2631,
           -0.1438])

In [ ]:
import torch.nn.functional as F
# Softmax is also in torch.nn.functional
data = torch.randn(5)
print(data)
print(F.softmax([data], dim=0))
print(F.softmax(data, dim=0).sum())  # Sums to 1 because it is a distribution!
print(F.log_softmax(data, dim=0))  # theres also log_softmax

In [ ]:
print(_input,"\n\n\n",tgt)